In [1]:
import pickle
import pandas as pd
import mysql.connector
from scipy.spatial import distance
import matplotlib.pyplot as plt
import numpy as np
import json
import math
import ast

In [2]:
Directory = ""

In [3]:
df_car_edge =pd.read_csv(Directory+'df_edge_car.csv')
df_ego_car_edge =pd.read_csv(Directory+'df_edge_ego_car.csv')
df_car =  pd.read_csv(Directory+ 'car_df.csv')
df_ego_car = pd.read_csv(Directory+ 'ego_car_df.csv')

In [ ]:
bin_size =5
df_car['equal_frequency_velocity_diff'] = pd.qcut(df_car['velocity_diff'], q=3, labels=False) + 1
df_car['equal_frequency_distance_difff'] = pd.qcut(df_car['distance_diff_level'], q=5, labels=False) + 1
df_car['equal_frequency_velocity_level'] = pd.qcut(df_car['velocity_abs'], q=3, labels=False) + 1

df_ego_car['equal_frequency_velocity_level'] = pd.qcut(df_ego_car['velocity_abs'], q=3, labels=False) + 1


df_car_edge['equal_frequency_velocity_diff'] = pd.qcut(df_car_edge['velocity_diff'], q=3, labels=False) + 1
df_car_edge['abs_distance_level'] = pd.qcut(df_car_edge['abs_distance'], q=5, labels=False) + 1



In [197]:
df_ego_car[['scene_id', 'frame_id']] = df_ego_car['id'].str.strip('()').str.split(',', expand=True)
df_car[['scene_id', 'frame_id']] = df_car['id'].str.strip('()').str.split(',', expand=True)

In [198]:
df_car = df_car[['Name', 'velocity_abs', 'lane_id', 'road_id',
       'left_blinker_on', 'right_blinker_on', 'brake_light_on', 'lane_idx',
       'orig_lane_idx', 'invading_lane', 'id',
       'velocity_abs_ego', 'velocity_ego', 'location_ego', 'rotation_ego',
       'abs_distance', 'velocity_diff', 'velocity_level',
       'velocity_diff_level', 'distance_diff_level',
       'equal_frequency_velocity_diff', 'equal_frequency_distance_difff',
       'equal_frequency_velocity_level', 'isInLane', 'isInego', 'index',
       'scene_id', 'frame_id']]

df_ego_car=df_ego_car[['velocity_abs', 'velocity', 'location',
       'rotation', 'ang_velocity', 'lane_id', 'road_id',
       'left_blinker_on', 'right_blinker_on', 'brake_light_on', 'lane_idx',
       'orig_lane_idx', 'invading_lane', 'id', 'velocity_level',
       'isInLane', 'scene_id','equal_frequency_velocity_level',
       'frame_id']]

df_ego_car['scene_id'] = df_ego_car['scene_id'].astype(int)
df_ego_car['frame_id'] = df_ego_car['frame_id'].astype(int)

df_car['scene_id'] = df_car['scene_id'].astype(int)
df_car['frame_id'] = df_car['frame_id'].astype(int)


In [199]:
#create connection to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="road_scene_2"
)
cursor = connection.cursor()

In [200]:
#table for cars
create_table_query = """
CREATE TABLE cars (
    car_id INT,
    PRIMARY KEY (car_id)
);
"""
cursor.execute(create_table_query)

In [201]:
create_table_query = """
CREATE TABLE ego_cars (
    ego_id INT,
    PRIMARY KEY (ego_id)
);
"""
cursor.execute(create_table_query)

In [202]:
#table for frames
create_table_query = """
CREATE TABLE frames (
    f_id VARCHAR(100) NOT NULL,
    PRIMARY KEY (f_id)
);
"""
cursor.execute(create_table_query)

In [203]:
#table for successive frames
create_table_query = """
CREATE TABLE succ_frame (
    f_id1 VARCHAR(100) NOT NULL,
    f_id2 VARCHAR(100) NOT NULL,
    PRIMARY KEY (f_id1,f_id2),
    FOREIGN KEY(f_id1)  REFERENCES frames(f_id),
    FOREIGN KEY(f_id2)  REFERENCES frames(f_id)
);
"""
cursor.execute(create_table_query)

In [204]:
#table for cars
create_table_query = """
CREATE TABLE ego_frame (
    f_id VARCHAR(100) NOT NULL,
    ego_id INT NOT NULL,
    Speed VARCHAR(11) NOT NULL,
    PRIMARY KEY (f_id,ego_id),
    FOREIGN KEY(f_id)  REFERENCES frames(f_id),
    FOREIGN KEY(ego_id)  REFERENCES ego_cars(ego_id)
);
"""
cursor.execute(create_table_query)

In [205]:
create_table_query = f"""
CREATE TABLE relation (
    f_id VARCHAR(100) NOT NULL,
    car_id INT NOT NULL,
    speed_diff VARCHAR(11),
    near_level VARCHAR(11),
    Lane VARCHAR(100) NOT NULL,
    Speed VARCHAR(11) NOT NULL,
    PRIMARY KEY (f_id, car_id),
    FOREIGN KEY(f_id)  REFERENCES frames(f_id),
    FOREIGN KEY(car_id)  REFERENCES cars(car_id)
)
"""
cursor.execute(create_table_query)
print(f"Table relation created successfully.")

Table relation created successfully.


In [206]:
scene_list = df_car['scene_id'].unique()


scene_list = scene_list

scene_set= []
car_dict = dict()
for x in scene_list:
    t_car_df = df_car[df_car['scene_id']==x]
    t_ego_df = df_ego_car[df_ego_car['scene_id']==x]
    frame_list =t_ego_df['frame_id'].tolist()
    car_dict[x] = (t_car_df['Name'].unique())
    frame_list.sort()
    succ = []
    pre= frame_list[0]
    for i in range(1,len(frame_list)):
        cur = frame_list[i]
        succ.append((x,pre,cur))
        pre = cur
    scene_set.append(succ)

In [207]:
df_car = df_car[df_car['scene_id'].isin(scene_list)]
df_ego_car = df_ego_car[df_ego_car['scene_id'].isin(scene_list)]
#df_car_edge= df_car_edge[df_car_edge['scene_id'].isin(scene_list)]

In [208]:
df_car['car_id'] = df_car.groupby(['scene_id', 'Name']).ngroup()

In [209]:
#load data into the cars table
table_name = 'cars' 
insert_list = 'car_name,car_id'
d_car = df_car
d_car= d_car.drop_duplicates(subset=['Name','scene_id'])

for index, row in d_car.iterrows():
    query = "INSERT INTO " + table_name + " (car_id) VALUES (%s)"    
    values= (str(row['car_id']),)
    cursor.execute(query,values)
connection.commit()


In [210]:
for x in df_ego_car['scene_id'].unique():
    query = "INSERT INTO ego_cars (ego_id) VALUES (" +str(x)+ ")"    
    cursor.execute(query)
connection.commit()

In [211]:
#load data into the frames table
table_name = 'frames'
insert_list = 'f_id'
for index,row in df_ego_car.iterrows():
    query = "INSERT INTO " + table_name + " ("+ insert_list+ ") VALUES ( %s)"
    values =((str(row['frame_id']))+'|'+str(row['scene_id']),)
    cursor.execute(query,values)  
connection.commit()

In [212]:
grouped = df_ego_car.groupby('scene_id')
grouped_dataframes = {}
for group_name, group_data in grouped:
    grouped_dataframes[group_name] = group_data


In [213]:
table_name = 'succ_frame'
insert_list = 'f_id1,f_id2'
frame_id_dict= dict()
for x in grouped_dataframes.keys():
    l= grouped_dataframes[x]['frame_id'].tolist()
    frame_id_dict[x]=sorted(l)

for x in frame_id_dict.keys(): 
    l =frame_id_dict[x]
    for i in range(1,len(l)):
        query = "INSERT INTO " + table_name + " ("+ insert_list+ ") VALUES ( %s,%s)"
        values = (str(l[i-1])+'|'+str(x),str(l[i])+'|'+str(x))
        cursor.execute(query,values)  
connection.commit()

In [214]:
table_name = 'ego_frame'
insert_list = 'f_id,ego_id,Speed'
for index, row in df_ego_car.iterrows():
    query = "INSERT INTO " + table_name + " ("+ insert_list+ ") VALUES (%s,%s,%s)"
    values = (str(row['frame_id'])+'|'+str(row['scene_id']),str(row['scene_id']),row['equal_frequency_velocity_level'])
    cursor.execute(query,values)  
connection.commit()

In [215]:
df_car

,Name,velocity_abs,lane_id,road_id,left_blinker_on,right_blinker_on,brake_light_on,lane_idx,orig_lane_idx,invading_lane,...,distance_diff_level,equal_frequency_velocity_diff,equal_frequency_distance_difff,equal_frequency_velocity_level,isInLane,isInego,index,scene_id,frame_id,car_id
0,car_6083,23,-2.0,46.0,True,False,True,1.0,curr,0.0,...,1,2,1,1,lane_middle,0,0,0,17794829,4
1,car_5820,1,-1.0,46.0,True,False,True,0.0,curr,0.0,...,8,1,4,1,lane_left,0,1,0,17794829,0
2,car_5845,17,-4.0,46.0,True,False,True,3.0,curr,0.0,...,10,2,5,1,lane_left,0,2,0,17794829,1
3,car_5874,30,-3.0,46.0,True,False,True,2.0,curr,0.0,...,1,3,1,2,lane_middle,0,3,0,17794829,2
4,car_6083,24,-2.0,46.0,True,False,True,1.0,curr,0.0,...,1,2,1,1,lane_middle,0,4,0,17794831,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131780,car_4132,51,6.0,50.0,True,False,True,3.0,curr,0.0,...,3,3,1,3,lane_middle,0,131780,270,30421907,2107
131781,car_4042,27,5.0,50.0,True,False,True,2.0,curr,0.0,...,5,1,2,2,lane_left,0,131781,270,30421908,2102
131782,car_4046,18,5.0,50.0,True,False,True,2.0,curr,0.0,...,4,1,2,1,lane_left,0,131782,270,30421908,2103
131783,car_4088,67,3.0,50.0,True,False,True,0.0,curr,0.0,...,5,3,2,3,lane_left,0,131783,270,30421908,2105


In [216]:
c_df = df_car_edge
merged_df = pd.merge(c_df, df_car[['id','Name','isInLane','frame_id','scene_id','velocity_level','car_id','equal_frequency_velocity_level']], left_on=['id', 'Node_1'],right_on=['id', 'Name'],suffixes=('','_car'))
merged_df = pd.merge(merged_df, df_ego_car[['id']], left_on=['id'],right_on=['id'],suffixes=('','_ego'))

In [217]:
for index, row in merged_df.iterrows():
    insert_query = "INSERT INTO relation (f_id, car_id,speed_diff,near_level,Lane,Speed) VALUES (%s,%s,%s,%s,%s,%s)"
    values = (str(row['frame_id'])+'|'+str(row['scene_id']), str(row['car_id']),
              str(row['equal_frequency_velocity_diff']),str(row['near_level']),row['isInLane'],row['equal_frequency_velocity_diff'])
    cursor.execute(insert_query, values)

connection.commit()
